# Understanding how the SQuAD dataset is set up for the text extraction task with BERT

We are going to fine-tune [BERT implemented by HuggingFace](https://huggingface.co/bert-base-uncased) for the text extraction task with a dataset of questions and answers with the [SQuAD (The Stanford Question Answering Dataset)](https://rajpurkar.github.io/SQuAD-explorer/) dataset.
The data is composed by a set of questions and corresponding paragraphs that contains the answers.
The model will be trained to locate the answer in the context by giving the positions where the answer starts and ends.

In this notebook we are going to see how the data is set up for training.

This notebook is based on [BERT (from HuggingFace Transformers) for Text Extraction](https://keras.io/examples/nlp/text_extraction_with_bert/).

More info:
- [Glossary - HuggingFace docs](https://huggingface.co/transformers/glossary.html#model-inputs)
- [BERT NLP — How To Build a Question Answering Bot](https://towardsdatascience.com/bert-nlp-how-to-build-a-question-answering-bot-98b1d1594d7b)

In [1]:
import os
import utility.data_processing as dpp
from datasets import load_dataset, load_metric
from transformers import BertTokenizer
from tokenizers import BertWordPieceTokenizer
from rich.pretty import pprint

/apps/daint/UES/6.0.UP04/sandboxes/sarafael/hpcpython2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets.utils import disable_progress_bar
from datasets import disable_caching


disable_progress_bar()
disable_caching()

## The raw data

In [3]:
bert_cache = os.path.join(os.getcwd(), 'cache')

In [4]:
hf_dataset = load_dataset('squad')

Dataset squad downloaded and prepared to /users/class424/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


In [5]:
hf_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
for i, _squad_example in enumerate(hf_dataset['train']):
    pprint(_squad_example)
    if i > 5:
        break

{
│   'id': '5733be284776f41900661182',
│   'title': 'University_of_Notre_Dame',
│   'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
│   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
│   'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}
}

{
│   'id': '5733be284776f4190066117f',
│   'title': 'University_of_Notre_Dame',
│   'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
│   'question': 'What is in front of the Notre Dame Main Building?',
│   'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}
}

{
│   'id': '5733be284776f41900661180',
│   'title': 'University_of_Notre_Dame',
│   'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
│   'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
│   'answers': {'text': ['the Main Building'], 'answer_start': [279]}
}

{
│   'id': '5733be284776f41900661181',
│   'title': 'University_of_Notre_Dame',
│   'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
│   'question': 'What is the Grotto at Notre Dame?',
│   'answers': {'text': ['a Marian place of prayer and reflection'], 'answer_start': [381]}
}

{
│   'id': '5733be284776f4190066117e',
│   'title': 'University_of_Notre_Dame',
│   'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
│   'question': 'What sits on top of the Main Building at Notre Dame?',
│   'answers': {'text': ['a golden statue of the Virgin Mary'], 'answer_start': [92]}
}

{
│   'id': '5733bf84d058e614000b61be',
│   'title': 'University_of_Notre_Dame',
│   'context': "As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.",
│   'question': 'When did the Scholastic Magazine of Notre dame begin publishing?',
│   'answers': {'text': ['September 1876'], 'answer_start': [248]}
}

{
│   'id': '5733bf84d058e614000b61bf',
│   'title': 'University_of_Notre_Dame',
│   'context': "As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals. Begun as a one-page journal in September 1876, the Scholastic magazine is issued twice monthly and claims to be the oldest continuous collegiate publication in the United States. The other magazine, The Juggler, is released twice a year and focuses on student literature and artwork. The Dome yearbook is published annually. The newspapers have varying publication interests, with The Observer published daily and mainly reporting university and other news, and staffed by students from both Notre Dame and Saint Mary's College. Unlike Scholastic and The Dome, The Observer is an independent publication and does not have a faculty advisor or any editorial oversight from the University. In 1987, when some students believed that The Observer began to show a conservative bias, a liberal newspaper, Common Sense was published. Likewise, in 2003, when other students believed that the paper showed a liberal bias, the conservative paper Irish Rover went into production. Neither paper is published as often as The Observer; however, all three are distributed to all students. Finally, in Spring 2008 an undergraduate journal for political science research, Beyond Politics, made its debut.",
│   'question': "How often is Notre Dame's the Juggler published?",
│   'answers': {'text': ['twice'], 'answer_start': [441]}
}

In [7]:
for i, _squad_example in enumerate(hf_dataset['validation']):
    pprint(_squad_example)
    if i > 5:
        break

{
│   'id': '56be4db0acb8001400a502ec',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'Which NFL team represented the AFC at Super Bowl 50?',
│   'answers': {
│   │   'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
│   │   'answer_start': [177, 177, 177]
│   }
}

{
│   'id': '56be4db0acb8001400a502ed',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'Which NFL team represented the NFC at Super Bowl 50?',
│   'answers': {
│   │   'text': ['Carolina Panthers', 'Carolina Panthers', 'Carolina Panthers'],
│   │   'answer_start': [249, 249, 249]
│   }
}

{
│   'id': '56be4db0acb8001400a502ee',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'Where did Super Bowl 50 take place?',
│   'answers': {
│   │   'text': [
│   │   │   'Santa Clara, California',
│   │   │   "Levi's Stadium",
│   │   │   "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."
│   │   ],
│   │   'answer_start': [403, 355, 355]
│   }
}

{
│   'id': '56be4db0acb8001400a502ef',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'Which NFL team won Super Bowl 50?',
│   'answers': {
│   │   'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'],
│   │   'answer_start': [177, 177, 177]
│   }
}

{
│   'id': '56be4db0acb8001400a502f0',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'What color was used to emphasize the 50th anniversary of the Super Bowl?',
│   'answers': {'text': ['gold', 'gold', 'gold'], 'answer_start': [488, 488, 521]}
}

{
│   'id': '56be8e613aeaaa14008c90d1',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'What was the theme of Super Bowl 50?',
│   'answers': {
│   │   'text': ['"golden anniversary"', 'gold-themed', '"golden anniversary'],
│   │   'answer_start': [487, 521, 487]
│   }
}

{
│   'id': '56be8e613aeaaa14008c90d2',
│   'title': 'Super_Bowl_50',
│   'context': 'Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi\'s Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.',
│   'question': 'What day was the game played on?',
│   'answers': {
│   │   'text': ['February 7, 2016', 'February 7', 'February 7, 2016'],
│   │   'answer_start': [334, 334, 334]
│   }
}

In [8]:
len(hf_dataset['train']['title'])

87599

In [9]:
len(hf_dataset['validation']['title'])

10570

In [10]:
len(set(hf_dataset['train']['title']))

442

In [11]:
len(set(hf_dataset['validation']['title']))

48

In [12]:
squad_ex = hf_dataset['train'].select([20584])

In [13]:
squad_ex['title']

['Alps']

In [14]:
squad_ex['context']

['The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".']

In [15]:
squad_ex['question']

['How long has it taken for the Alps to form? ']

In [16]:
squad_ex['answers']

[{'text': ['over tens of millions of years'], 'answer_start': [475]}]

# The tokenizer

## Processing the data for training
Now we process the data so we can feed it later to the model.
The idea is to replace the words (and some word parts) by numbers using the tokenizer above and organize the training data as a set of paragraphs and questions.

In [17]:
hf_model = 'bert-base-uncased'

slow_tokenizer = BertTokenizer.from_pretrained(
    hf_model,
    cache_dir=os.path.join(bert_cache, f'_{hf_model}-tokenizer'),
)

Downloading: 100%|██████████| 226k/226k [00:00<00:00, 557kB/s] 
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 36.2kB/s]
Downloading: 100%|██████████| 570/570 [00:00<00:00, 544kB/s]


In [18]:
# a faster tokenizer implementation
save_path = os.path.join(bert_cache, f'{hf_model}-tokenizer')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    slow_tokenizer.save_pretrained(save_path)
    
# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer(os.path.join(save_path, 'vocab.txt'),
                                   lowercase=True)

In [19]:
encoding = tokenizer.encode("Let's tokenize something?")

In [20]:
encoding.tokens

['[CLS]', 'let', "'", 's', 'token', '##ize', 'something', '?', '[SEP]']

In [21]:
encoding.ids

[101, 2292, 1005, 1055, 19204, 4697, 2242, 1029, 102]

In [22]:
tokenizer.decode(encoding.ids)

"let's tokenize something?"

In [23]:
for i, j in encoding.offsets:
    print("Let's tokenize something?"[i: j])


Let
'
s
token
ize
something
?



## Processing the data

In [24]:
max_len = 384

In [25]:
hf_dataset.flatten()

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers.text', 'answers.answer_start'],
        num_rows: 10570
    })
})

In [26]:
%%time
processed_dataset = hf_dataset.flatten().map(
    lambda example: dpp.process_squad_item_batched(example, max_len, tokenizer),
    remove_columns=hf_dataset.flatten()['train'].column_names,
    batched=True,  # dpp.process_squad_item_batched needs `batched=True`
    num_proc=12
)

CPU times: user 604 ms, sys: 121 ms, total: 725 ms
Wall time: 9.09 s


In [27]:
processed_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_token_idx', 'end_token_idx'],
        num_rows: 86136
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_token_idx', 'end_token_idx'],
        num_rows: 34010
    })
})

In [28]:
train_dataset = processed_dataset["train"]
train_dataset.set_format(type='numpy')

# eval_dataset = processed_dataset["validation"]
# eval_dataset.set_format(type='torch')

### The SquadExample objects

In [29]:
squad_ex   # Alps

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 1
})

In [30]:
squad_ex_obj = dpp.create_squad_example(squad_ex[0], max_len, tokenizer)
type(squad_ex_obj)

utility.data_processing.SquadExample

In [31]:
squad_ex_obj.__dict__.keys()

dict_keys(['question', 'context', 'start_char_idx', 'answer_text', 'max_len', 'skip', 'tokenizer', 'input_ids', 'token_type_ids', 'attention_mask', 'start_token_idx', 'end_token_idx', 'context_token_to_char'])

## The training set

In [32]:
train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_token_idx', 'end_token_idx'],
    num_rows: 86136
})

In [33]:
train_sample = train_dataset.select([20299])[0]
pprint(train_sample)

{
│   'input_ids': array([  101,  1996, 13698,  1006,  1013,  1097, 14277,  2015,  1013,
│   │   1025,  3059,  1024,  2632,  8197,  1031,  1149,  2389,  8197,
│   │   1033,  1025,  2413,  1024,  2632, 10374,  1031,  2632,  2361,
│   │   1033,  1025,  2446,  1024,  2632, 11837,  1031,  1149, 29705,
│   │   2389,  9737,  1033,  1025, 18326,  1024,  2632,  5051,  1031,
│   │   1149,  2050, 23432, 14277, 29275,  1033,  1007,  2024,  1996,
│   │   3284,  1998,  2087,  4866,  3137,  2846,  2291,  2008,  3658,
│   │   4498,  1999,  2885,  1010, 10917,  3155,  1015,  1010,  3263,
│   │   3717,  1006,  9683,  2771,  1007,  2408,  2809, 10348,  3032,
│   │   1024,  5118,  1010,  2605,  1010,  2762,  1010,  3304,  1010,
│      26500,  1010, 14497,  1010, 10307,  1010,  1998,  5288,  1012,
│   │   1996, 16512,  4020,  2024,  3020,  1010,  1998,  1996, 24471,
│   │   9777,  2936,  1010,  2021,  2119,  4682,  6576,  1999,  4021,
│   │   1012,  1996,  4020,  2020,  2719,  2058, 15295,  1997,  8817,
│   │   1997,  2086,  2004,  1996,  3060,  1998, 23399,  8915, 28312,
│   │   2594,  7766, 17745,  1012,  6034,  2460,  7406,  3303,  2011,
│   │   1996,  2724,  4504,  1999,  3884, 25503,  5749,  4803,  2011,
│      21468,  1998, 12745,  2046,  2152,  3137, 11373,  2107,  2004,
│      18318, 18698,  1998,  1996,  3043,  9769,  1012, 18318, 18698,
│      14798,  1996,  2413,  1516,  3059,  3675,  1010,  1998,  2012,
│   │   1018,  1010,  6282,  2692,  1049,  1006,  2321,  1010,  6275,
│   │   2487,  3027,  1007,  2003,  1996,  3284,  3137,  1999,  1996,
│      13698,  1012,  1996, 10348,  2555,  2181,  3397,  2055,  1037,
│   │   3634, 11373,  3020,  2084,  1018,  1010,  2199,  1049,  1006,
│   │   2410,  1010, 13138,  3027,  1007,  1010,  2124,  2004,  1996,
│   │   1000,  2176,  1011,  4595,  2545,  1000,  1012,   102,  2129,
│   │   2146,  2038,  2009,  2579,  2005,  1996, 13698,  2000,  2433,
│   │   1029,   102,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0,     0,     0,     0,
│   │      0,     0,     0,     0,     0,     0], dtype=int32),
│   'token_type_ids': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
│      1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
│     

## The model input

In [34]:
(
    train_sample['input_ids'].shape,
    train_sample['token_type_ids'].shape,
    train_sample['attention_mask'].shape
)

((384,), (384,), (384,))

In [35]:
train_sample['input_ids']

array([  101,  1996, 13698,  1006,  1013,  1097, 14277,  2015,  1013,
        1025,  3059,  1024,  2632,  8197,  1031,  1149,  2389,  8197,
        1033,  1025,  2413,  1024,  2632, 10374,  1031,  2632,  2361,
        1033,  1025,  2446,  1024,  2632, 11837,  1031,  1149, 29705,
        2389,  9737,  1033,  1025, 18326,  1024,  2632,  5051,  1031,
        1149,  2050, 23432, 14277, 29275,  1033,  1007,  2024,  1996,
        3284,  1998,  2087,  4866,  3137,  2846,  2291,  2008,  3658,
        4498,  1999,  2885,  1010, 10917,  3155,  1015,  1010,  3263,
        3717,  1006,  9683,  2771,  1007,  2408,  2809, 10348,  3032,
        1024,  5118,  1010,  2605,  1010,  2762,  1010,  3304,  1010,
       26500,  1010, 14497,  1010, 10307,  1010,  1998,  5288,  1012,
        1996, 16512,  4020,  2024,  3020,  1010,  1998,  1996, 24471,
        9777,  2936,  1010,  2021,  2119,  4682,  6576,  1999,  4021,
        1012,  1996,  4020,  2020,  2719,  2058, 15295,  1997,  8817,
        1997,  2086,

In [36]:
tokenizer.decode(train_sample['input_ids'])

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

## [Attention masks](https://huggingface.co/transformers/glossary.html#attention-mask)
To create batches for training the text needs to be padded. The attention masks differentiate what is text and what is padding.

In [37]:
train_sample['attention_mask']

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [38]:
context_encoded = train_sample['input_ids'][train_sample['attention_mask'] == 1]
tokenizer.decode(context_encoded)

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ". ho

## [Token type ids](https://huggingface.co/transformers/glossary.html#token-type-ids)
Differentiate two types of tokens, the ones that correspond to the question and the ones that correspond to the answers.

In [39]:
train_sample['token_type_ids']

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [40]:
paragraph_encoded = train_sample['input_ids'][train_sample['token_type_ids'] == 0]
tokenizer.decode(paragraph_encoded)

'the alps ( / ælps / ; italian : alpi [ ˈalpi ] ; french : alpes [ alp ] ; german : alpen [ ˈʔalpm ] ; slovene : alpe [ ˈaːlpɛ ] ) are the highest and most extensive mountain range system that lies entirely in europe, stretching approximately 1, 200 kilometres ( 750 mi ) across eight alpine countries : austria, france, germany, italy, liechtenstein, monaco, slovenia, and switzerland. the caucasus mountains are higher, and the urals longer, but both lie partly in asia. the mountains were formed over tens of millions of years as the african and eurasian tectonic plates collided. extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as mont blanc and the matterhorn. mont blanc spans the french – italian border, and at 4, 810 m ( 15, 781 ft ) is the highest mountain in the alps. the alpine region area contains about a hundred peaks higher than 4, 000 m ( 13, 123 ft ), known as the " four - thousanders ".'

In [41]:
question_encoded = train_sample['input_ids'][train_sample['token_type_ids'] == 1]
tokenizer.decode(question_encoded)

'how long has it taken for the alps to form?'

### The references

In [42]:
train_sample['start_token_idx'], train_sample['end_token_idx']

(122, 128)

In [43]:
print('\n * CONTEXT:                   \n', squad_ex_obj.context)
print('\n * QUESTION:                  \n', squad_ex_obj.question)
print('\n * ANSWER (REFERENCE):        \n', squad_ex_obj.answer_text[0])
print('\n * ANSWER FROM CONTEXT:       \n', tokenizer.decode(train_sample['input_ids'][train_sample['start_token_idx']:
                                                                                       train_sample['end_token_idx']]))
print('\n\n === TRAINING SAMPLE ===')
print('\n * CONTEXT & QUESTION:        \n', tokenizer.decode(train_sample['input_ids']))
print('\n * POSITION in CONTEXT:       \n', (train_sample['start_token_idx'], train_sample['end_token_idx']))


 * CONTEXT:                   
 The Alps (/ælps/; Italian: Alpi [ˈalpi]; French: Alpes [alp]; German: Alpen [ˈʔalpm̩]; Slovene: Alpe [ˈáːlpɛ]) are the highest and most extensive mountain range system that lies entirely in Europe, stretching approximately 1,200 kilometres (750 mi) across eight Alpine countries: Austria, France, Germany, Italy, Liechtenstein, Monaco, Slovenia, and Switzerland. The Caucasus Mountains are higher, and the Urals longer, but both lie partly in Asia. The mountains were formed over tens of millions of years as the African and Eurasian tectonic plates collided. Extreme shortening caused by the event resulted in marine sedimentary rocks rising by thrusting and folding into high mountain peaks such as Mont Blanc and the Matterhorn. Mont Blanc spans the French–Italian border, and at 4,810 m (15,781 ft) is the highest mountain in the Alps. The Alpine region area contains about a hundred peaks higher than 4,000 m (13,123 ft), known as the "four-thousanders".

 * QUE